<a href="https://colab.research.google.com/github/667029/KVP10k/blob/main/LayoutLMv3_KVP10k_GG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IKKE KJØR DENNE NOTEBOOKEN :)

_______

In [ ]:
!pip install -q transformers datasets seqeval

In [ ]:
!pip install evaluate


In [ ]:
import os              #navigere mapper og filer, hente filbaner
from PIL import Image  #åpne, vise og manipulere bilder
import json            #lese/skrive til JSON-filer
from transformers import LayoutLMv3Processor
import torch
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
base_path = "/content/drive/MyDrive/DAT255/KVP10k-dataset/kvp10k/"
print(os.listdir(base_path))

In [ ]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False) # <-- Viktig fordi vi allerede har utført OCR på bildet og har tekst og bboxes

In [ ]:
# Mapping fra tekstlige BIO-labels til tall som modellen bruker
label_map = {
    "O": 0,
    "B-KEY": 1,
    "I-KEY": 2,
    "B-VALUE": 3,
    "I-VALUE": 4,
}

# Funksjon for å skalere bounding boxes til 0-1000 (som LayoutLMv3 krever)
def normalize_bbox(bbox, width, height):
  return [
      int(1000 * (bbox[0] /width)),
      int(1000 * (bbox[1] / height)),
      int(1000 * (bbox[2] / width)),
      int(1000 * (bbox[3] / height))
  ]


def assign_label_for_box(box, boxes, label_type):
  """Returnerer liste med (index, label) for tokens som overlapper box"""
  overlaps = []
  for i, token_box in enumerate(boxes):
    if box_overlap(box, token_box) > 0:
      overlaps.append(i)

  overlaps = sorted(overlaps)

  labeled = []
  for j, idx in enumerate(overlaps):
    tag = f"B-{label_type}" if j == 0 else f"I-{label_type}"
    labeled.append((idx, tag))

  return labeled


#Sjekker om OCR-boksen overlapper med GTS(key/value)-boksen.
#Ved overlapp hører de til hverandre.
def box_overlap(box1, box2):
  x0 = max(box1[0], box2[0])
  y0 = max(box1[1], box2[1])
  x1 = min(box1[2], box2[2])
  y1 = min(box1[3], box2[3])
  return max(0, x1 - x0) * max(0, y1 - y0)


# Funksjon for å generere BIO-labels fra gts (ground truth).
# Lager en BIO-label for hvert token basert på om det overlapper med en key- eller value-boks fra GTS.
# Matcher hvert token fra OCR (word + bbox) mot key/value-bbokser fra gts:
# --> Token overlapper en nøkkelboks: B-KEY eller I-KEY
# --> Token overlapper en verdiboks: B-VALUE eller I-VALUE
# --> Ellers: O
def iob_from_kvps(words, boxes, kvps):
  labels = ["O"] * len(words)

  #Gå igjennom alle key-value-pairs
  for kvp in kvps:
    if "key" in kvp and "bbox" in kvp["key"]:
      key_bbox = kvp["key"]["bbox"]
      for idx, tag in assign_label_for_box(key_bbox, boxes, "KEY"):
        labels[idx] = tag

    if "value" in kvp and "bbox" in kvp["value"]:
      value_box = kvp["value"]["bbox"]
      for idx, tag in assign_label_for_box(value_box, boxes, "VALUE"):
        labels[idx] = tag

  return labels

In [ ]:
!ls -lh /content/drive/MyDrive/KVP10k_processed_ready


In [ ]:
from google.colab import drive
from datasets import load_from_disk

# Monter Drive (hvis du ikke har gjort det)
drive.mount("/content/drive", force_remount=True)

# Last inn dataset fra riktig path
dataset = load_from_disk("/content/drive/MyDrive/KVP10k_processed_ready")

# Hent splits
train_dataset = dataset["train"]
eval_dataset = dataset["eval"]
test_dataset = dataset["test"]

# (Valgfritt) Gjør klar for huggingface Trainer
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

# Sjekk at alt fungerer
print("✅ Datasett lastet inn!")
print("Train:", len(train_dataset))
print("Eval:", len(eval_dataset))
print("Test:", len(test_dataset))
print("Keys:", train_dataset[0].keys())

In [ ]:
train_dataset.features

In [ ]:
example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

In [ ]:
processor.tokenizer.decode(train_dataset[0]["input_ids"])

In [ ]:
for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())

In [ ]:
from evaluate import load
metric = load("seqeval")

In [ ]:
import numpy as np

label_list = ["O", "B-KEY", "I-KEY", "B-VALUE", "I-VALUE"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

In [ ]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="test",
    num_train_epochs=8,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=5e-6,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=125,
    save_steps=250,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs",
    logging_steps=50,
    report_to="tensorboard",
    lr_scheduler_type="cosine",
    warmup_steps=100,
    fp16=True
)



In [ ]:
from transformers import Trainer, EarlyStoppingCallback
from transformers.data.data_collator import default_data_collator

trainer = Trainer (
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.evaluate(test_dataset)


In [ ]:
# Angi en mappe i Drive (eller lokalt hvis du vil kopiere senere)
output_dir = "/content/drive/MyDrive/layoutlmv3_kvp10k_model"

# Lagre modell og tokenizer
#trainer.save_model(output_dir)
#processor.save_pretrained(output_dir)  # dette lagrer både tokenizer + feature extracto

_________
INFERENCE

In [ ]:
from transformers import AutoProcessor, AutoModelForTokenClassification
from google.colab import drive
drive.mount('/content/drive')


model_path = "/content/drive/MyDrive/layoutlmv3_kvp10k_model"

# Last inn modellen (med dine finetunede vekter)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# Last inn processor (inneholder både tokenizer og feature extractor)
processor = AutoProcessor.from_pretrained(model_path)

# Sett modellen til riktig device (valgfritt, men vanlig)
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

id2label = model.config.id2label
id2label = model.config.id2label
label_map = label2id

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

def unnormalize_box(bbox, width, height):
    return [
        width * (bbox[0] / 1000),
        height * (bbox[1] / 1000),
        width * (bbox[2] / 1000),
        height * (bbox[3] / 1000),
    ]

def predict_and_visualize(doc_id, show_gt=True):
    base_path = "/content/drive/MyDrive/DAT255/KVP10k-dataset/kvp10k/test"

    # === Last inn bilde og metadata
    image_path = f"{base_path}/images/{doc_id}.png"
    ocr_path = f"{base_path}/ocrs/{doc_id}.json"
    gt_path = f"{base_path}/gts/{doc_id}.json"

    image = Image.open(image_path).convert("RGB")
    with open(ocr_path, "r", encoding="utf-8") as f:
        ocr_data = json.load(f)
    with open(gt_path, "r", encoding="utf-8") as f:
        gt_data = json.load(f)

    # === Hent tekst og bokser
    page = ocr_data["pages"][0]
    words = [w["text"] for w in page["words"]]
    raw_boxes = [w["bbox"] for w in page["words"]]
    width, height = page["width"], page["height"]
    norm_boxes = [normalize_bbox(b, width, height) for b in raw_boxes]

    # === Lag word_labels fra GT
    string_labels = iob_from_kvps(words, raw_boxes, gt_data["kvps_list"])
    word_labels = [label_map[l] for l in string_labels]

    # === Encoding for modellen
    encoding = processor(
        image,
        words,
        boxes=norm_boxes,
        word_labels=word_labels,
        return_tensors="pt",
        truncation=True,
        padding="max_length"
    )
    inputs = {k: v.to(model.device) for k, v in encoding.items()}

    # === Modellprediksjon
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    input_ids = encoding["input_ids"].squeeze().tolist()
    labels = encoding["labels"].squeeze().tolist()
    bboxes = encoding["bbox"].squeeze().tolist()
    predictions = outputs.logits.argmax(-1).squeeze().tolist()

    # === Unnormalize bboxes
    unnorm_boxes = [unnormalize_box(b, width, height) for b in bboxes]
    tokens = [processor.tokenizer.decode([tid]) for tid in input_ids]

    # === Filtrer vekk padding og spesialtokens
    filtered = [
        (token, id2label[label], id2label[pred], box)
        for token, label, pred, box in zip(tokens, labels, predictions, unnorm_boxes)
        if label != -100 and token not in ["[PAD]", "[CLS]", "[SEP]"]
    ]

    # === Tegn prediksjoner
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    def iob_to_label(label):
        return label[2:].lower() if label.startswith(("B-", "I-")) else "other"

    label2color = {
        "key": "blue",
        "value": "green",
        "other": "gray"
    }

    for token, true, pred, box in filtered:
        if pred == "O":
            continue
        label = iob_to_label(pred)
        draw.rectangle(box, outline=label2color.get(label, "red"), width=2)
        draw.text((box[0] + 5, box[1] - 10), label, fill=label2color.get(label, "red"), font=font)

    print("📷 Modellens prediksjoner:")
    display(image)

    # === Fasit (valgfritt)
    if show_gt:
        gt_img = Image.open(image_path).convert("RGB")
        draw_gt = ImageDraw.Draw(gt_img)

        for word, box, label_id in zip(words, raw_boxes, string_labels):
            if label_id == "O":
                continue
            label_type = iob_to_label(label_id)
            draw_gt.rectangle(box, outline=label2color.get(label_type, "gray"), width=2)
            draw_gt.text((box[0] + 5, box[1] - 10), label_type, fill=label2color.get(label_type, "gray"), font=font)

        print("✅ Ground Truth:")
        display(gt_img)


In [ ]:
#predict_and_visualize("aaf643426f0250efd10de3d9df63b407292f3fcc2aa335e399c37aca32443ea1")
#predict_and_visualize("aaed61e79aa3edbae844f5775789ebb6aa1a94a23d9cb3468d2cfc974af304e5")
predict_and_visualize("aa35720ba3611f946c372cc99d8cd1d78e81265b8ceb51dcdb4672d196944c2b")


_______